# 1D harmonic oscillator 
Rosa Kurtz

In [2]:
# MPL
import matplotlib        as mpl
import matplotlib.pyplot as plt

# NumPy
import numpy as np
from numpy import exp, log, sqrt, pi, sin, cos

# SciPy
import scipy.optimize as optimize

# Settings
mpl.style.use('default')
mpl.rcParams['legend.frameon'] = False

enter_w = input("enter w (radial frequency): ")
w=float(enter_w)

m = 1/2
hbar = 1
x0 = -10
xN = 10
xM = (xN + x0) // 2
N = 500

h = (xN - x0)/N

def correct():
    check = np.zeros([20])
    for i in range(0, len(check)):
        check[i] = hbar * w * (1/2 + i)
    return check

def V(x):
    return .5 * m * (w ** 2) * (x ** 2)

##### Numerov Method
def funct_f(i, E, x0, h):
    x = i * h + x0
    return (2 * m / (hbar ** 2)) * (V(x) - E)

def Numerov(P, E, x0, h):
    for i in range (2, len(P)):
        front_coeff = 1 - (1/12) * (h**2) * funct_f(i, E, x0, h)
        mid_coeff = 1 - (1/12) * (h**2) * funct_f(i-1, E, x0, h)
        back_coeff = 1 - (1/12) * (h**2) * funct_f(i-2, E, x0, h)
        P[i] = ((2 * mid_coeff * P[i-1]) + (h**2 * funct_f(i,E, x0, h) * P[i-1]) - (back_coeff * P[i-2])) / front_coeff
    return 
#####

def normalize(P_I, P_II):
    sum = P_I[0]**2 + P_II[0]**2 #+ 4 * P_I[0]**2 + 4 * P_II[0]**2
    for i in range (1, len(P_I)):
        sum = sum + 2*(i%2 + 1) * (P_I[i]**2 + P_II[i]**2)
    sum = sqrt(sum * h/3)
    #print(sum)
    return P_I/sum, P_II/sum

##### Shooting Method
def Match_Method(E_lo, E_hi):
    x_I = np.linspace(x0, xM, N//2)
    x_II = np.linspace(xN, xM, N//2)
    P_I = np.zeros([N//2]) 
    P_I[0] = 0
    P_I[1] = 1
    P_II = np.zeros([N//2])
    P_II[0] = 0
    P_II[1] = 1
    
    h1 = (xM - x0) / (N // 2)
    h2 = (xM - xN) / (N // 2)

    Numerov(P_I, E_lo, x0, h1 )
    Numerov(P_II, E_hi, xN, h2)

    s = P_I[-1] / P_II[-1]

    for i in range(0, len(P_II)):
        P_II[i] = P_II[i] * s
    
    c = ((2 * m) / (hbar**2)) #just 1
    P_I, P_II = normalize(P_I, P_II)
    
    del1 = ((P_I[-2] + P_II[-2] - P_I[-1] * 2))/h - (h * c * (V(xM) - E_lo) * P_I[-1])
    del2 = ((P_I[-2] + P_II[-2] - P_I[-1] * 2))/h - (h * c * (V(xM) - E_hi) * P_I[-1])
    
    return del1, del2, x_I, x_II, P_I, P_II


def bisection(f, x1, x2, x3):
    # Set limits
    accuracy = 1E-9
    Nmax = 1000
    
    # Iterate
    i = 0
    f1, f2, x_I1, x_II1, P_I1, P_II1 = f(x1, x2)
    fa, fb, x_I2, x_II2, P_I2, P_II2 = f(x2, x3)
    # Init and check
    while (abs(f1 - f2) > accuracy and abs(fa - fb) > accuracy) and i < Nmax:
        f1, f2, x_I1, x_II1, P_I1, P_II1 = f(x1, x2)
        fa, fb, x_I2, x_II2, P_I2, P_II2 = f(x2, x3)

        i += 1
        if(abs(f1 * f2) < abs(fa * fb)):
            x3 = x2
            x2 = (x1 + x2)/2
        else: 
            x1 = x2
            x2 = (x2 + x3)/2   
    
    d1, d2, x_I, x_II, P_I, P_II = Match_Method(x2, x2)
    return x2, i, x_I, x_II, P_I, P_II
####

def energy_states():
    En = np.zeros([20])
    en = 0;
    
    for i in range(0, len(En)):
        n1=i*w*hbar
        n2=(i+1)*w*hbar
        xc, iterations, x_I, x_II, P_I, P_II = bisection(Match_Method, n1, (n1+n2)/2, n2)
        En[i] = xc
    
    print("here are the calculated energy eigenvalues: ")    
    print(En)   
    
    return 


print()
print("this is a check, here are the correct energy eigenvalues: ")
print(correct())
print()
energy_states()



enter w (radial frequency): 9

this is a check, here are the correct energy eigenvalues: 
[  4.5  13.5  22.5  31.5  40.5  49.5  58.5  67.5  76.5  85.5  94.5 103.5
 112.5 121.5 130.5 139.5 148.5 157.5 166.5 175.5]

here are the calculated energy eigenvalues: 
[  4.49908717  13.49917603  22.49687274  31.49807739  40.49286682
  49.49588013  58.48755527  67.49313354  76.48101407  85.48983765
  94.47323372 103.48544312 112.46416792 121.47994995 130.45375025
 139.47418213 148.44190159 157.46621704 166.42853407 175.45742798]
